In [ ]:
import pandas as pd

chunk_size = 1000

file_path = '../replay_data_public.WOE.PremierDraft.csv'

# Initialize a dictionary to store mana usage patterns and win counts
mana_usage_patterns = {}

# Function to check mana usage for a single player
def classify_mana_usage(row, player_prefix, max_turns):
    pattern = []
    for i in range(1, max_turns + 1):
        mana_spent = row[f'user_turn_{i}_{player_prefix}_mana_spent'] + row[f'oppo_turn_{i}_{player_prefix}_mana_spent']
        if mana_spent < i:
            pattern.append(f'did not maximize mana on turn {i}')
    return ', '.join(pattern) if pattern else 'Maximized mana every turn'

# Create an iterator to read in chunks
chunk_iter = pd.read_csv(file_path, chunksize=chunk_size)

for chunk in chunk_iter:
    for index, row in chunk.iterrows():
        max_turns = min(row['num_turns'], 5)  # Consider up to turn 5 or num_turns, whichever is smaller

        # Classify for user
        user_pattern = classify_mana_usage(row, 'user', max_turns)
        if user_pattern not in mana_usage_patterns:
            mana_usage_patterns[user_pattern] = {'wins': 0, 'total': 0}
        mana_usage_patterns[user_pattern]['total'] += 1
        if row['won']:
            mana_usage_patterns[user_pattern]['wins'] += 1

        # Classify for opponent
        oppo_pattern = classify_mana_usage(row, 'oppo', max_turns)
        if oppo_pattern not in mana_usage_patterns:
            mana_usage_patterns[oppo_pattern] = {'wins': 0, 'total': 0}
        mana_usage_patterns[oppo_pattern]['total'] += 1
        if not row['won']:
            mana_usage_patterns[oppo_pattern]['wins'] += 1
    

# Calculate win rates for each pattern
for pattern, data in mana_usage_patterns.items():
    data['win_rate'] = data['wins'] / data['total'] if data['total'] > 0 else 0

# Convert the results to a DataFrame for display
results_df = pd.DataFrame.from_dict(mana_usage_patterns, orient='index')

pd.set_option('display.max_rows', None)  # Set to None to display all rows
print(results_df)
pd.reset_option('display.max_rows')
